## Pandas Exercise

### Part 1: Aggregation, Group By and Sort Review

Below is a dataframe, with values as a list-of-lists and columns as a list.

In [1]:
import pandas as pd

df = pd.DataFrame([[123,'xt23',20],
                   [123,'q45',2],
                   [123,'a89',25],
                   [77,'q45',3],
                   [77,'a89',30],
                   [92,'xt23',24],
                   [92,'m33',60],
                   [92,'a89',28]], columns=['userid','product','price'])
df

,userid,product,price
0,123,xt23,20
1,123,q45,2
2,123,a89,25
3,77,q45,3
4,77,a89,30
5,92,xt23,24
6,92,m33,60
7,92,a89,28


**Q: We want the maximum price anyone has paid.**

In [12]:
df.describe()
#$60

,userid,price
count,8.000000,8.000000
mean,99.875000,24.000000
std,20.131266,18.087091
min,77.000000,2.000000
25%,88.250000,15.750000
50%,92.000000,24.500000
75%,123.000000,28.500000
max,123.000000,60.000000


**Q: We want the maximum price per user. Hint: Use groupby.**

In [70]:
df.groupby('userid')['price'].max()

userid
77     30
92     60
123    25
Name: price, dtype: int64

**Q: What is the total amount paid by user?**

Note: Pandas will smartly leave out columns for which that aggregation doesn't have meaning.

In [17]:
df.groupby('userid').sum()

,price
userid,
77,33
92,112
123,47


**Q: Sort the dataframe by userid first, and then price, both ascending.**

In [34]:
df_sort = df.sort_values(by=['userid','price'],ascending = True)
df_sort

,userid,product,price
3,77,q45,3
4,77,a89,30
5,92,xt23,24
7,92,a89,28
6,92,m33,60
1,123,q45,2
0,123,xt23,20
2,123,a89,25


### Part 2: Combining Techniques

**Q: We want the maximum price each user paid, and the product associated with that price.**

Note for SQL users: In SQL, you groupby and then sort, but in pandas, it's easier to do it the other way around.

In [44]:
df.groupby('userid').max()

,product,price
userid,,
77,q45,30
92,xt23,60
123,xt23,25


### Part 3: Exploring The Index

Let's add a new column.

In [45]:
df['website'] = ['Amazon', 'Amazon', 'NewEgg', 'NewEgg',
                 'NewEgg', 'Amazon', 'Amazon', 'Amazon']
df

,userid,product,price,website
0,123,xt23,20,Amazon
1,123,q45,2,Amazon
2,123,a89,25,NewEgg
3,77,q45,3,NewEgg
4,77,a89,30,NewEgg
5,92,xt23,24,Amazon
6,92,m33,60,Amazon
7,92,a89,28,Amazon


**Q: What is the total amount paid by each user on each website?**

In [52]:
df.groupby(['userid','website']).sum()

price
userid website       
77     NewEgg      33
92     Amazon     112
123    Amazon      22
       NewEgg      25

Now let's do the same groupby as above, but set the as_index flag to "False". This will result in a flat table instead of the nested indexes.

In [57]:
df.groupby(['userid', 'website'], as_index = False).sum()

,userid,website,price
0,77,NewEgg,33
1,92,Amazon,112
2,123,Amazon,22
3,123,NewEgg,25


### Part 4: Merging

Let's now create a second table:

In [74]:
df2 = pd.DataFrame([
    [123, 'USA'], [77, 'Canada'], [92, 'USA']],
    columns=['userid', 'country'])
df2

,userid,country
0,123,USA
1,77,Canada
2,92,USA


We can combine the two tables using a merge function. What it does is, it will do a pairwise comparision of every row in table1 with every row in table2 and if the "on" condition matches, it will create a single row with columns from both those matched rows.

Merge of two tables with 5 rows each can give as little as 0 rows and as much as 25 rows.

    [1,2,3,4,5] merged with [6,7,8,9,10] will give 0 rows
    [1,2,3,4,5] merged with [1,2,3,4,5] will give 5 rows
    [1,1,1,1,1] merged with [1,1,1,1,1] will give 25 rows

In [62]:
df_merge pd.merge(df, df2, on='userid')
df_merge

,userid,product,price,website,country
0,123,xt23,20,Amazon,USA
1,123,q45,2,Amazon,USA
2,123,a89,25,NewEgg,USA
3,77,q45,3,NewEgg,Canada
4,77,a89,30,NewEgg,Canada
5,92,xt23,24,Amazon,USA
6,92,m33,60,Amazon,USA
7,92,a89,28,Amazon,USA


**Q: What is the total amount paid per country?**

In [68]:
df_merge.groupby('country')['price'].sum()

country
Canada     33
USA       159
Name: price, dtype: int64

**Q: What is the average amount paid per country and website?**

In [72]:
df_merge.groupby(['country','website'],as_index=False)['price'].mean()

,country,website,price
0,Canada,NewEgg,16.5
1,USA,Amazon,26.8
2,USA,NewEgg,25.0


### Part 5: The Final Question - A Demo

Let's add another column: purchase date

In [73]:
df['date'] = ['2018-01-12',
              '2018-01-08',
              '2018-01-06',
              '2018-01-03',
              '2018-01-05',
              '2018-01-04',
              '2018-01-07',
              '2018-01-02']
df

,userid,product,price,website,date
0,123,xt23,20,Amazon,2018-01-12
1,123,q45,2,Amazon,2018-01-08
2,123,a89,25,NewEgg,2018-01-06
3,77,q45,3,NewEgg,2018-01-03
4,77,a89,30,NewEgg,2018-01-05
5,92,xt23,24,Amazon,2018-01-04
6,92,m33,60,Amazon,2018-01-07
7,92,a89,28,Amazon,2018-01-02


**Q: Here is a tricky task. For each row, I want the average purchase price for that user prior to that purchase.**

One option is to do some loops. But another solution is to just do a merge on itself and filter.

### Movie / Wine Example

But first, quick question:

Say you merge (or 'join' if you come from SQL) two dataframes with 3 rows each, how many rows would you end up with?

Could be anything between 0-9.

Consider the following examples, where table x has users and the movies they like. And table y has users and the wines they line. And let's do a merge to come up with possible movie and wine pairings for any user. In case A, we get 0 rows, in case B, we get 3 rows and case C we get 9 rows.

#### Merge Two Tables with No IDs in Common

In [75]:
dfx = pd.DataFrame([[1,'Godfather'],[2,'Amelie'],[3,'Chicago']],columns=['userid','movies'])
dfx

,userid,movies
0,1,Godfather
1,2,Amelie
2,3,Chicago


In [76]:
dfy = pd.DataFrame([[4,'red'],[5,'white'],[6,'pink']],columns=['userid','wines'])
dfy

,userid,wines
0,4,red
1,5,white
2,6,pink


In [77]:
dfm = pd.merge(dfx,dfy,on='userid')
dfm

,userid,movies,wines


#### Merge Two Tables with IDs in Common (1 Value Per ID)

In [78]:
dfx = pd.DataFrame([[1,'Godfather'],[2,'Amelie'],[3,'Chicago']],columns=['userid','movies'])
dfx

,userid,movies
0,1,Godfather
1,2,Amelie
2,3,Chicago


In [79]:
dfy = pd.DataFrame([[1,'red'],[2,'white'],[3,'pink']],columns=['userid','wines'])
dfy

,userid,wines
0,1,red
1,2,white
2,3,pink


In [80]:
dfm = pd.merge(dfx,dfy,on='userid')
dfm

,userid,movies,wines
0,1,Godfather,red
1,2,Amelie,white
2,3,Chicago,pink


#### Merge Two Tables with IDs in Common (Multiple Values Per ID)

In [81]:
dfx = pd.DataFrame([[1,'Godfather'],[1,'Amelie'],[1,'Chicago']],columns=['userid','movies'])
dfx

,userid,movies
0,1,Godfather
1,1,Amelie
2,1,Chicago


In [82]:
dfy = pd.DataFrame([[1,'red'],[1,'white'],[1,'pink']],columns=['userid','wines'])
dfy

,userid,wines
0,1,red
1,1,white
2,1,pink


In [83]:
dfm = pd.merge(dfx,dfy,on='userid')
dfm

,userid,movies,wines
0,1,Godfather,red
1,1,Godfather,white
2,1,Godfather,pink
3,1,Amelie,red
4,1,Amelie,white
5,1,Amelie,pink
6,1,Chicago,red
7,1,Chicago,white
8,1,Chicago,pink


### Back to the Question

Now let's return to the original question: For each row, I want the average purchase price for that user prior to that purchase. Let's do a merge on itself and filter.

Here are the steps we're going to take:
1. **MERGE**: Join table on itself. For each userid / date combo, show me all userid / date / price combos.
1. **FILTER**: For each userid / date combo, keep only the userid / date / price combos that were from earlier. Filter everything else out.
1. **AGGREGATE**: For each userid / date combo, find the average price for the remaining rows.
1. **MERGE**: Combine these values with the original dataframe for the final result.
1. **SORT**: Sort to make the results look pretty.

#### MERGE: Join table on itself. For each userid / date combo, show me all userid / date / price combos.

In [84]:
df_date = df[['userid','date']]
df_date

,userid,date
0,123,2018-01-12
1,123,2018-01-08
2,123,2018-01-06
3,77,2018-01-03
4,77,2018-01-05
5,92,2018-01-04
6,92,2018-01-07
7,92,2018-01-02


In [85]:
df_all = df[['userid','price','date']]
df_all

,userid,price,date
0,123,20,2018-01-12
1,123,2,2018-01-08
2,123,25,2018-01-06
3,77,3,2018-01-03
4,77,30,2018-01-05
5,92,24,2018-01-04
6,92,60,2018-01-07
7,92,28,2018-01-02


In [86]:
df2 = pd.merge(df_date, df_all, on='userid')
df2

,userid,date_x,price,date_y
0,123,2018-01-12,20,2018-01-12
1,123,2018-01-12,2,2018-01-08
2,123,2018-01-12,25,2018-01-06
3,123,2018-01-08,20,2018-01-12
4,123,2018-01-08,2,2018-01-08
5,123,2018-01-08,25,2018-01-06
6,123,2018-01-06,20,2018-01-12
7,123,2018-01-06,2,2018-01-08
8,123,2018-01-06,25,2018-01-06
9,77,2018-01-03,3,2018-01-03


#### FILTER: For each userid / date combo, keep only the userid / date / price combos that were from earlier. Filter everything else out.

In [87]:
df3 = df2[df2['date_x'] > df2['date_y']]
df3

,userid,date_x,price,date_y
1,123,2018-01-12,2,2018-01-08
2,123,2018-01-12,25,2018-01-06
5,123,2018-01-08,25,2018-01-06
11,77,2018-01-05,3,2018-01-03
15,92,2018-01-04,28,2018-01-02
16,92,2018-01-07,24,2018-01-04
18,92,2018-01-07,28,2018-01-02


#### **AGGREGATE**: For each userid / date combo, find the average price for the remaining rows.

In [88]:
df4 = df3.groupby(['userid','date_x'])[['price']].mean()
df4.rename(columns={'price': 'avg_prior_price'}, inplace=True)
df4

avg_prior_price
userid date_x                     
77     2018-01-05              3.0
92     2018-01-04             28.0
       2018-01-07             26.0
123    2018-01-08             25.0
       2018-01-12             13.5

#### **MERGE**: Combine these values with the original dataframe for the final result.

In [89]:
df

,userid,product,price,website,date
0,123,xt23,20,Amazon,2018-01-12
1,123,q45,2,Amazon,2018-01-08
2,123,a89,25,NewEgg,2018-01-06
3,77,q45,3,NewEgg,2018-01-03
4,77,a89,30,NewEgg,2018-01-05
5,92,xt23,24,Amazon,2018-01-04
6,92,m33,60,Amazon,2018-01-07
7,92,a89,28,Amazon,2018-01-02


In [90]:
df4.index

MultiIndex([( 77, '2018-01-05'),
            ( 92, '2018-01-04'),
            ( 92, '2018-01-07'),
            (123, '2018-01-08'),
            (123, '2018-01-12')],
           names=['userid', 'date_x'])

In [91]:
final = df.merge(df4,
                 left_on=['userid','date'],
                 right_on=['userid','date_x'],
                 right_index=True,
                 how='left')
final

,userid,product,price,website,date,avg_prior_price
0,123,xt23,20,Amazon,2018-01-12,13.5
1,123,q45,2,Amazon,2018-01-08,25.0
2,123,a89,25,NewEgg,2018-01-06,NaN
3,77,q45,3,NewEgg,2018-01-03,NaN
4,77,a89,30,NewEgg,2018-01-05,3.0
5,92,xt23,24,Amazon,2018-01-04,28.0
6,92,m33,60,Amazon,2018-01-07,26.0
7,92,a89,28,Amazon,2018-01-02,NaN


#### **SORT**: Sort to make the results look pretty.

In [92]:
final.sort_values(by=['userid','date'])

,userid,product,price,website,date,avg_prior_price
3,77,q45,3,NewEgg,2018-01-03,NaN
4,77,a89,30,NewEgg,2018-01-05,3.0
7,92,a89,28,Amazon,2018-01-02,NaN
5,92,xt23,24,Amazon,2018-01-04,28.0
6,92,m33,60,Amazon,2018-01-07,26.0
2,123,a89,25,NewEgg,2018-01-06,NaN
1,123,q45,2,Amazon,2018-01-08,25.0
0,123,xt23,20,Amazon,2018-01-12,13.5
